## Troll and Toad Card Price Scraper

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
test_data = {"Card_Name": ["Aegislash", "Pikachu and Zekrom GX", "Darkrai VSTAR"],
             "Card_Number": ["177/185", "SM248", "GG50/GG70"]}
test_df = pd.DataFrame(test_data)
test_df

,Card_Name,Card_Number
0,Aegislash,177/185
1,Pikachu and Zekrom GX,SM248
2,Darkrai VSTAR,GG50/GG70


In [3]:
card_urls = []

In [4]:
for ind in test_df.index:
    card_query = ""
    card_words = test_df.Card_Name[ind].split()
    for word in card_words:
        card_query += "{}+".format(word)
    if "/" in test_df.Card_Number[ind]:
        card_number, set_number = test_df.Card_Number[ind].split("/")
        card_query += "{}%2F{}".format(card_number, set_number)
    else:
        card_query += "{}".format(test_df.Card_Number[ind])
    url = "https://www.trollandtoad.com/category.php?selected-cat=0&search-words={}".format(card_query)
    card_urls.append(url)

In [5]:
card_page = requests.get(card_urls[0])
if card_page.status_code == 200:
    print("Successfully Loaded Page")
else:
    print("Unsuccessful")

Successfully Loaded Page


In [6]:
parsed_page = bs(card_page.text, "html.parser")

In [7]:
print(parsed_page.find_all("a",  class_ = "card-text"))

[<a class="card-text" href="/pokemon/sword-shield-vivid-voltage-singles/aegislash-v-177-185-full-art-ultra-rare/1680812"><strong>Aegislash</strong> V - <strong>177/185</strong> - Full Art Ultra Rare</a>]
